# 집값 예측 경진대회 베이스라인

이번 대회는 집의 면적, 상태, 퀄리티 등의 다양한 정보들을 통해 집값을 예측하는 대회입니다.

언뜻보면 간단해 보이지만 처음 데이터 분석을 접하시는 분들에게는 어디부터 어떻게 시작해야 하는지 막막하실 겁니다.

## 누구나 할 수 있는 데이터 분석

딥러닝, 머신러닝, 모델, 학습, 회귀분석 .... 다 몰라도 괜찮습니다!

누구나 할 수 있는 방식으로 간단하게 데이터 분석 & 예측을 진행해보겠습니다.

# 데이터 불러오기

먼저 분석하려는 데이터를 작업장으로 가져오는 작업이 필요합니다.

이를 위해서 파이썬 라이브러리 중 하나인 Pandas를 이용합니다.

pandas 라이브러리는 엑셀과 같은 행과 열로 이루어진 테이블(table) 형식의 데이터를 편하게 다루도록 해주는 라이브러리입니다.

데이터 분석에는 반드시 사용된다고 해도 과언이 아닌 라이브러리이니, 시간이 날때마다 공부해둡시다.

In [1]:
import pandas as pd

# csv 형식으로 된 데이터 파일을 읽어옵니다.
train = pd.read_csv('train.csv')

print(f'train set은 {train.shape[1]} 개의 feature를 가진 {train.shape[0]} 개의 데이터 샘플로 이루어져 있습니다.')

# 데이터의 최상단 5 줄을 표시합니다.
train.head()

train set은 15 개의 feature를 가진 1350 개의 데이터 샘플로 이루어져 있습니다.


,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000


데이터를 자세히 살펴보는 것은 EDA 글을 통해 더 자세히 배워봅시다.

여기에서는 분석 방법에 초점을 맞추겠습니다.

## 결측치 확인

결측치(NA: Not Available)란 값이 누락된 데이터를 말합니다.

보다 정확한 분석을 하기 위해서는 데이터의 결측치를 확인하고 적절히 처리해주어야 합니다.

이번 데이터에 결측치가 있나 확인해볼까요?

In [4]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


이번 데이터에는 결측치가 존재하지 않네요.

만약 결측치가 존재한다면 결측치 삭제, 평균 보간법, 최빈값 보간법 등 다양한 방식으로 처리할 수 있습니다.

# 데이터 설명 방정식 만들기

데이터 분석이란, 주어진 데이터를 분석하여 데이터들 사이의 관계를 알아내고,

이 관계를 통해서 주어지지 않은 데이터를 예측하는 일입니다.

이 말을 잘 생각해보면 데이터 분석이란 방정식을 푸는 일과 똑같다는 걸 알 수 있습니다.

X, Y를 이용해서 만드는 가장 기본적인 방정식을 떠올려봅시다.

## Y = aX + b

여기서 X는 주어진 데이터를 의미하며, Y는 예측하려는 데이터를 의미합니다.

이번 대회에서는{ X란 [차고 면적, 건축 연도, 집의 퀄리티 등]이며, Y는 [집의 가격]}입니다.

우리의 목표는 X에 어떤 a를 곱하고, 어떤 b를 더해야 정확하게 Y를 구할 수 있는지, 그 a와 b를 찾는 것입니다.

먼저 완전히 주어진 데이터인 train.csv를 이용하여 데이터들 사이의 관계를 파악해봅시다.

## 좀 더 단순하게

먼저 문제를 단순하게 생각해봅시다.

X = {[Gr Liv Area]} 만을 이용해서 

Y = {[SalePrice]} 를 예측해봅시다.

In [5]:
# feature를 확인합니다.
train.columns

Index(['id', 'Overall Qual', 'Gr Liv Area', 'Exter Qual', 'Garage Cars',
       'Garage Area', 'Kitchen Qual', 'Total Bsmt SF', '1st Flr SF',
       'Bsmt Qual', 'Full Bath', 'Year Built', 'Year Remod/Add',
       'Garage Yr Blt', 'target'],
      dtype='object')

In [2]:
# X와 Y를 각각 따로 저장합니다.
x_feature = 'Gr Liv Area'
y_feature = 'target'

X = train[x_feature]
Y = train[y_feature]

X

0       2392
1       1352
2        900
3       1174
4       1958
        ... 
1345    1756
1346    2748
1347    1214
1348     894
1349     907
Name: Gr Liv Area, Length: 1350, dtype: int64

## a 구하기

먼저 X와 Y가 어떤 관계를 갖는지, 그 평균 값을 구해봅니다.

이 과정은 위 방정식의 a 값을 구하는 것으로 생각할 수 있습니다.

In [3]:
import numpy as np

# 각각의 평균을 구합니다.
X_mean = np.mean(X)
Y_mean = np.mean(Y)

# 평균값을 이용하여 관계를 구합니다.
relation = Y_mean/X_mean

print(f"{x_feature}의 평균:{X_mean:.2f} \n{y_feature}의 평균:{Y_mean:.2f}")
print(f"따라서 {y_feature}는 {x_feature}의 평균 {relation:.2f}배 입니다.")
print(f"즉, a = {relation:.2f} 입니다.")
a = round(relation, 2)
print(f"업데이트 된 방정식 : Y = {a:.2f} * X + b")

Gr Liv Area의 평균:1513.54 
target의 평균:186406.31
따라서 target는 Gr Liv Area의 평균 123.16배 입니다.
즉, a = 123.16 입니다.
업데이트 된 방정식 : Y = 123.16 * X + b


## b 구하기

이번에는 b를 구해봅시다.

a는 X와 Y의 강력한 관계를 나타냈다면, b는 그보다는 약한 관계를 나타냅니다. 

a 만으로는 설명하지 못하는 부분을 구하는 과정이라고 볼 수 있습니다.

a를 넣어서 새롭게 업데이트 된 방정식을 사용하면 b를 구할 수 있습니다.

b = Y - a * X

In [4]:
# b = Y - a * X 를 그대로 작성합니다. 여기서도 평균값을 이용합니다.
b = Y_mean - a * X_mean

print(f"b는 {b:.2f} 입니다.")
print(f"업데이트 된 방정식 : Y = {a:.2f} * X + {b:.2f}")

b는 -1.55 입니다.
업데이트 된 방정식 : Y = 123.16 * X + -1.55


In [5]:
predict_Y = a * X + b
predict_Y

0       294597.172504
1       166510.772504
2       110842.452504
3       144588.292504
4       241145.732504
            ...      
1345    216267.412504
1346    338442.132504
1347    149514.692504
1348    110103.492504
1349    111704.572504
Name: Gr Liv Area, Length: 1350, dtype: float64

# 예측 결과 평가하기

예측값을 만들었다고 데이터 분석이 끝나지는 않습니다.

우리가 예측한 값이 얼마나 정확한지 평가하는 과정을 통해 어떤 부분이 부족한지, 

어떤 부분을 개선해야 더 정확한 예측이 가능한지 살펴봐야 합니다.

### NMAE

이번 대회의 평가 지표(metric)는 NMAE, Normalized Mean Absolute Error 입니다.

NMAE는 우리가 예측한 값과 실제 값이 얼마나 차이가 나는지, 정규화된 오차의 평균의 평균 (NMAE) 값을 취한 것입니다.

다시 말하면, 정규화된(Normalized) 실제 값과 예측 값의 차이(Error)를 절대값(Absolute)으로 변환해 평균화(Mean)한 것이 NMAE 입니다. 

단순 실제값과 예측값의 차이는 음수 또는 양수 모두 나올 수 있어 에러의 크기를 그대로 반영하기 위해 절댓값을 취해준 것입니다.

In [6]:
# 대회 규칙의 평가 산식 함수를 그대로 사용합니다.

def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [7]:
# 실제 정답 값을 준비합니다.
real_answer = Y.copy()

# 정답과 예측 값을 함수에 넣어 결과를 확인합니다.
error = NMAE(real_answer, predict_Y)

print(f"이 방정식의 평균 에러는 {error:.2f} 입니다.")

이 방정식의 평균 에러는 0.20 입니다.


# test_data 예측하기

지금까지 train.csv를 통해서 데이터들 사이의 관계를 파악했습니다.

그 결과 X로 Y를 예측하는 방정식을 얻을 수 있었습니다.

이제 이 관계를 이용해서 실제로 주어지지 않은 데이터를 예측해봅시다.

In [9]:
# 우리가 예측하고자 하는 test 데이터를 불러옵니다.
test = pd.read_csv('test.csv')

test.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt
0,1,9,1800,Gd,2,702,Ex,1800,1800,Ex,2,2007,2007,2007
1,2,6,1082,TA,1,240,TA,1082,1082,TA,1,1948,1950,1948
2,3,6,1573,Gd,2,440,Gd,756,769,Gd,2,2000,2000,2000
3,4,6,2443,Gd,3,744,Gd,1158,1158,Gd,2,2004,2004,2004
4,5,5,1040,TA,2,686,TA,1040,1040,TA,1,1968,1968,1991


In [10]:
# 예측의 재료를 가져옵니다.
test_X = test[x_feature]

# 예측을 진행합니다.
predict_test = a * test_X + b

# 예측 결과를 확인합니다.
predict_test

0       221686.452504
1       133257.572504
2       193729.132504
3       300878.332504
4       128084.852504
            ...      
1345    237943.572504
1346    112320.372504
1347    106039.212504
1348    176117.252504
1349    287823.372504
Name: Gr Liv Area, Length: 1350, dtype: float64

In [11]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,target
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [12]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission[y_feature] = predict_test

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,target
0,1,221686.452504
1,2,133257.572504
2,3,193729.132504
3,4,300878.332504
4,5,128084.852504
...,...,...
1345,1346,237943.572504
1346,1347,112320.372504
1347,1348,106039.212504
1348,1349,176117.252504


In [13]:
# submission을 csv 파일로 저장합니다.
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다. 
# 정확한 채점을 위해 꼭 index=False를 넣어주세요.
submission.to_csv("baseline.csv", index=False)

# 축하합니다! 데이터 분석을 완료하셨습니다!

데이터 분석의 첫걸음을 떼신 것을 축하드립니다.

보셨다시피 데이터 분석은 그렇게 어렵지 않습니다.

데이터들 사이의 관계를 찾아서, 그 관계를 통해 주어지지 않은 정보를 예측할 뿐입니다.

고도의 기술을 사용하면 이 예측값이 조금 더 정확해질뿐, 기본적인 개념은 바뀌지 않습니다.

앞으로도 데이콘과 함께 즐겁게 데이터 분석 능력을 키워가시면 좋겠습니다.

감사합니다.